In [ ]:
#####Installing required libraries

# !pip install requests
# !pip install xlsxwriter

requests library provides access to websites.
xlsxwriter library provides tools to read, write and modify the spreadsheets.

In [ ]:
#####Rest of your code remains unchanged
from bs4 import BeautifulSoup
import pandas as pd
import requests
from datetime import datetime
import time
import random
import gspread
from requests.exceptions import ConnectionError

'Beautifulsoup' : provides tools to navigate through the html code and extracting data from it.
'pandas' : provides tools to make and modify dataframes to store the data.
'requests' : provides access to websites.
'xlsxwriter' : provides tools to read, write and modify the spreadsheets.
'datetime' : used to change the format of date to our standard format.
'time' : used for countdown (in delay functions).
'random' : used for generating a random number from given range (in delay functions).
'gspread' : used to read, write and modify the google sheets (using it's API).
'ConnectionError' : used to indentify any error occured during runtime.


In [ ]:
#####Google sheets API setup for the notebook
from oauth2client.service_account import ServiceAccountCredentials
scope = ["https://spreadsheets.google.com/feeds",'https://www.googleapis.com/auth/spreadsheets',"https://www.googleapis.com/auth/drive.file","https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("creds2020.json",scope)
client = gspread.authorize(creds)

'ServiceAccountCredentials' : provides access to service accounts made for using API
scope, contains various links required for service account to access the API and bridges output to sheet
creds, is the json files that contains all data of service account with a gmail and a key.
client, act as a mediator that authorises our credentials from the creds file.

In [ ]:
#####Setup of required worksheet and cell number 

#var1 = client.open("Your_Sheet_Name")
#var2 = var1.get_worksheet(i) 
#var2_var1.update_acell('cell_no.' , 'Any_text')
sheet = client.open("TOI2020")
target_sheet = sheet.get_worksheet(2) #here "i" is index of your worksheet
target_sheet.update_acell('A1' , 'DateXXX')

Indexing of worksheets
Sheet 1 = 0 , Sheet 2 = 1 , Sheet 3 = 2
We can also use name of the worksheet ('name')

In [ ]:
#####Forming Dataframe to store and represent Data

#var3 = ['Column 1' , 'Column 2']
#var4 = pd.DataFrame(columns = var3)
my_columns = ['Date', 'Heading', 'Text']
final_dataframe1 = pd.DataFrame(columns=my_columns)


#####Day loop

New_cell = 'A1' #var_cell = 'your_target_cell'
x = 44927 #Here x represent the range for all article published in a year (here is for 2023)
index = 0 #index is the number of articles scrapped till now (will update in the terminal/output window 
Batch = 0 #Batch is the days upto which articles are scrapped
t = x + Batch
limit = 45291 #limit is the range upto which article of a single year are published (here is for 2023)

while t <= 45291:
    #cooldown period of 10 sec, so that website doesnot get a flooded by requests and eventually aborts the connection
    time.sleep(10)
    #this is the master link to access any article. Every article has a unique number which is equal to {t} in the master link
    daily_url = f'https://timesofindia.indiatimes.com/2021/1/1/archivelist/year-2021,month-1,starttime-{t}.cms'
    Batch += 1
    t += 1

    #####Retrieving links from the daily_url    
    response = requests.get(daily_url) #Makes a request to url for access
    html_content = response.text #Convert all html to text
    soup = BeautifulSoup(html_content, 'html.parser') #navigating through different html elements
    
    #####Locating html element with required data
    link = soup.find('tr', class_="rightColWrap") #All links for articles published in a single day are stored in a table with class "rightColWrap"
    links = link.find_all('a') #Accessing all anchor tags to get links 

    ###### using try and except to avoid any runtime error
    try : 
        try:
            #checking status of url response, 200 means connection successful
            response = requests.get(url)
            print("=" + str(response))
        
        except:
            #  Retrieving data from dataset of all links of article of same day
            local_url = ""
            for url in links:
                index += 1
                local_url = url.get('href') #to use link text as working hyperlink, otherwise it will comes out as a simple text
                response_link = requests.get(local_url) #Makes a request to url for access
                html_content = response_link.text #Convert all html to text
                soup_link = BeautifulSoup(html_content, 'html.parser') #navigating through different html elements

                try:
                    #using to avoid, broken links
                    test = 1
                    
            #### Extracting Heading
                    headings = [] #Defining an empty list
                    var = soup_link.find('h1').text #finding all h1 present in the link (well, each link will have only one h1)
                    Heading = var

            #### Extracting Date
                    datex = soup_link.find('div', class_='VEOUU').text  #var = var.find('div' , class_ = 'VEOUU').text /// Date is mentioned in this element
                    i = 18  # i = 18, necessary for required format 
                    date = "" # defining an empty string
                    
                    # will print the whole date, which lies in this range only
                    while (i < len((datex))):
                        date = date + datex[i]
                        i += 1

                    date_str = date # Converting Date from January 1, 2021 to 01-01-2021
                    date_obj = datetime.strptime(date_str, " %B %d, %Y") #this will tell variable about the location of different elements of date. %B = January
                    formatted_date_str = date_obj.strftime("%d-%m-%Y") #arranging the date elements in required format
                    Date = formatted_date_str  # Output: 01-01-2021

            #### Extracting Text
                    # var = var.find('div' , class_ = 'class_name').text /// extraxting the text mentioned in this elements only.
                    txt = soup_link.find('div', class_='_s30J clearfix').text
                    Text = txt
                except:
            #### If broken link is detected. (this will just pass that link)
                    test = 0
                    index -= 1
                    print("broken Link detected")

            #### Adding all scraped data in a dataframe
                # Using test, as it will only fill the data from correct working links
                if (test):
                    final_dataframe1.loc[index] = [
                        Date,
                        Heading,
                        Text
                    ]
                print("Batch - " + str(Batch) + "," + "Entries - " + str(index))

            #### Updating our dataframe to the google worksheet
                if (New_cell == 'A1'):
                    target_sheet.update(New_cell , [final_dataframe1.columns.values.tolist() ] + final_dataframe1.fillna(-1).values.tolist())
                else:
                    target_sheet.update(New_cell , [final_dataframe1.columns.values.tolist() ] + final_dataframe1.fillna(-1).values.tolist())
                print('DataFrame is updated to Excel File successfully.')


    except (requests.ConnectionError, requests.Timeout) as e:
        #### If, pc get disconnected from the Internet, then this will save us from unnecessary errors 
        print("Error:", e)
        print(e)
        print("Connection lost. Retrying in 100 seconds...")
        time.sleep(100)  # Add a 10-second delay before retrying
        try : 
            try:
                response = requests.get(url)
                print("=" + str(response))
            
            except:
                #  Retrieving data from links
                local_url = ""
                for url in links:
                    index += 1
                    local_url = url.get('href')
                    response_link = requests.get(local_url)
                    html_content = response_link.text
                    soup_link = BeautifulSoup(html_content, 'html.parser')

                    try:
                        test = 1
                        # Extracting Heading
                        headings = []
                        for tag in soup_link.find_all(['h1']):
                            headings.append(tag.text.strip())

                        sorted_headings = sorted(headings)
                        for h1 in sorted_headings:
                            Heading = h1

                        # Extracting Date
                        datex = soup_link.find('div', class_='VEOUU').text
                        i = 18
                        date = ""
                        while (i < len((datex))):
                            date = date + datex[i]
                            i += 1

                        # Converting Date from January 1, 2021 to 01-01-2021
                        date_str = date
                        date_obj = datetime.strptime(date_str, " %B %d, %Y")
                        formatted_date_str = date_obj.strftime("%d-%m-%Y")
                        Date = formatted_date_str  # Output: 01-01-2021

                        # Extracting Text
                        txt = soup_link.find('div', class_='_s30J clearfix').text
                        Text = txt
                    except:
                        test = 0
                        index -= 1
                        print("broken Link detected")

                    # Adding all scraped data in a dataframe
                    if (test):
                        final_dataframe1.loc[index] = [
                            Date,
                            Heading,
                            Text
                        ]
                    run += 1
                    print("Batch - " + str(Batch) + "," + "Entries - " + str(index) + "," + "Run - " + str(run))
                    
                    if (New_cell == 'A1'):
                        target_sheet.update(New_cell , [final_dataframe1.columns.values.tolist() ] + final_dataframe1.fillna(-1).values.tolist())
                    else:
                        target_sheet.update(New_cell , [final_dataframe1.columns.values.tolist() ] + final_dataframe1.fillna(-1).values.tolist())
                    print('DataFrame is updated to Excel File successfully.')

        except (requests.ConnectionError, requests.Timeout) as e:
            print("Again Error:", e)
            print(e)
            print("Connection lost. Retrying in another 100 seconds...")
            time.sleep(600)  # Add a another 600-second delay before retrying

            #  Retrieving data from links
            local_url = ""
            for url in links:
                            index += 1
                            local_url = url.get('href')
                            response_link = requests.get(local_url)
                            html_content = response_link.text
                            soup_link = BeautifulSoup(html_content, 'html.parser')

                            try:
                                test = 1
                                # Extracting Heading
                                headings = []
                                for tag in soup_link.find_all(['h1']):
                                    headings.append(tag.text.strip())

                                sorted_headings = sorted(headings)
                                for h1 in sorted_headings:
                                    Heading = h1

                                # Extracting Date
                                datex = soup_link.find('div', class_='VEOUU').text
                                i = 18
                                date = ""
                                while (i < len((datex))):
                                    date = date + datex[i]
                                    i += 1

                                # Converting Date from January 1, 2021 to 01-01-2021
                                date_str = date
                                date_obj = datetime.strptime(date_str, " %B %d, %Y")
                                formatted_date_str = date_obj.strftime("%d-%m-%Y")
                                Date = formatted_date_str  # Output: 01-01-2021

                                # Extracting Text
                                txt = soup_link.find('div', class_='_s30J clearfix').text
                                Text = txt
                            except:
                                test = 0
                                index -= 1
                                print("broken Link detected")

                            # Adding all scraped data in a dataframe
                            if (test):
                                final_dataframe1.loc[index] = [
                                    Date,
                                    Heading,
                                    Text
                                ]
                            run += 1
                            print("Batch - " + str(Batch) + "," + "Entries - " + str(index) + "," + "Run - " + str(run))
                            
                            if (New_cell == 'A1'):
                                target_sheet.update(New_cell , [final_dataframe1.columns.values.tolist() ] + final_dataframe1.fillna(-1).values.tolist())
                            else:
                                target_sheet.update(New_cell , [final_dataframe1.columns.values.tolist() ] + final_dataframe1.fillna(-1).values.tolist())
                            print('DataFrame is updated to Excel File successfully.')



print('DataFrame is written to Excel File successfully.')

Possible cases of Broken Links : Article removed, Link not working, Article is from any other publisher(other website)

two nested try-except are used to avoid error due to 'connection lost'. First, will wait to 100 seconds for pc to reconnect and it will move forward if connection is established again. Second, will wait for another 600 seconds for connection to establish.